# Multi-GPU Computing with [CuPy](https://cupy.chainer.org/) (Exercise)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cupy as cp
from cupy.cuda import Device
from timers import cupy_timer, cpu_timer

In [ ]:
mandelbrot_kernel = cp.ElementwiseKernel('float64 X, float64 Y, int32 itermax, float64 radius2',
                                         'int32 mandelbrot',
                                         '''
                                         mandelbrot = 0;
                                         double cx = X, cy = Y;
                                         double x = cx, y = cy;
                                         double xtemp;
                                         int nit = 0;
                                         while (x * x + y * y < radius2 && nit < itermax) {
                                             xtemp = x * x - y * y + cx;
                                             y = 2.0 * x * y + cy;
                                             x = xtemp;
                                             nit += 1;
                                         }
                                         mandelbrot = nit;''', 'mandelbrot_kernel')

In [ ]:
xmin, xmax = -2.0, 1.0
ymin, ymax = -1.0, 1.0
with cupy_timer(True) as timer:
    X, Y = cp.meshgrid(cp.linspace(xmin , xmax, 5000), cp.linspace(ymin, ymax, 5000))
    mandelbrot_array = mandelbrot_kernel(X, Y, 100, 4.0)
    
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)
mandelbrot_array = mandelbrot_array.get()
ax.imshow(np.log(1 + mandelbrot_array), extent=[xmin, xmax, ymin, ymax]);
ax.set_aspect('equal')
ax.set_ylabel('Im[c]')
ax.set_xlabel('Re[c]');

#### <mark>Solution</mark> Do the same computation by splitting the work between the GPUs

#### Split by columns

In [ ]:
xmin, xmax = -2.0, 1.0
ymin, ymax = -1.0, 1.0

rows = 5000
cols = 5000
total_gpus = 4
mandelbrot_arrays = [None] * total_gpus 
local_cols = cols // 4
for i in range(total_gpus):
    with Device(i):
        xmin_local = xmin + (xmax - xmin) / total_gpus * i
        xmax_local = xmin_local + (xmax - xmin) / total_gpus
        X, Y = cp.meshgrid(cp.linspace(xmin_local , xmax_local, local_cols), cp.linspace(ymin, ymax, rows))
        
        # Add some values (i * 10) related to the column, to be able to show different color for each gpu
        mandelbrot_arrays[i] = mandelbrot_kernel(X, Y, 100, 4.0) + i * 10


array = cp.hstack(mandelbrot_arrays).get()
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)
ax.imshow(np.log(1 + array), extent=[xmin, xmax, ymin, ymax]);
ax.set_aspect('equal')
ax.set_ylabel('Im[c]')
ax.set_xlabel('Re[c]');

#### Split by rows

In [ ]:
xmin, xmax = -2.0, 1.0
ymin, ymax = -1.0, 1.0

rows = 5000
cols = 5000
total_gpus = 4
mandelbrot_arrays = [None] * total_gpus 
local_rows = rows // 4
streams = []
with cpu_timer(True) as timer:
    for i in range(total_gpus):
        with Device(i):
            with cp.cuda.Stream() as s:
                ymin_local = ymin + (ymax - ymin) / total_gpus * i
                ymax_local = ymin_local + (ymax - ymin) / total_gpus
                X, Y = cp.meshgrid(cp.linspace(xmin , xmax, cols), cp.linspace(ymin_local, ymax_local, local_rows))

                # Add some values (i * 10) related to the row, to be able to show different color for each gpu
                mandelbrot_arrays[i] = mandelbrot_kernel(X, Y, 100, 4.0) + i * 10
                streams.append(s)
    
    for s in streams:
        s.synchronize()
        
array = cp.vstack(mandelbrot_arrays).get()
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)
ax.imshow(np.log(1 + array), extent=[xmin, xmax, ymin, ymax]);
ax.set_aspect('equal')
ax.set_ylabel('Im[c]')
ax.set_xlabel('Re[c]');

#### Split by rows and columns

In [ ]:
xmin, xmax = -2.0, 1.0
ymin, ymax = -1.0, 1.0

rows = 5000
cols = 5000
gpus_x = 2
gpus_y = 2
mandelbrot_arrays = [[None, None], [None, None]]
streams = []

with cpu_timer(True) as timer:
    for r in range(gpus_y):
        for c in range(gpus_x):
            with Device(r * gpus_y + c):
                with cp.cuda.Stream() as s:
                    ymin_local = ymin + (ymax - ymin) / gpus_y * r
                    ymax_local = ymin_local + (ymax - ymin) / gpus_y
                    xmin_local = xmin + (xmax - xmin) / gpus_x * c
                    xmax_local = xmin_local + (xmax - xmin) / gpus_x
                    X, Y = cp.meshgrid(cp.linspace(xmin_local , xmax_local, cols // gpus_x), cp.linspace(ymin_local, ymax_local, rows//gpus_y))

                    # Add some values (r * 10 + c * 20) related to the row and column, to be able to show different color for each gpu
                    mandelbrot_arrays[r][c] = mandelbrot_kernel(X, Y, 100, 4.0) + r * 10 + c * 20
                    streams.append(s)
    
    for s in streams:
        s.synchronize()


array = cp.vstack([cp.hstack([mandelbrot_arrays[j][i].get() for i in range(gpus_x)]) for j in range(gpus_y)]).get()
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111)
ax.imshow(np.log(1 + array), extent=[xmin, xmax, ymin, ymax]);
ax.set_aspect('equal')
ax.set_ylabel('Im[c]')
ax.set_xlabel('Re[c]');